# BIBLIOTEQUES ET LIBRAIRIES NECESSAIRES

In [1]:
#import scrapy
import os
import time
import json
import spacy
import pandas
import socket
import requests
from nerd import ner
from langdetect import detect
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.firefox import GeckoDriverManager

# VARIABLES

In [3]:
PAGE_INITIALE = "https://www.presidence.sn/actualites/discours"

In [4]:
methaDataNameFile = "InformationSurLesDiscroutEnregister.json"
linkDiscourtSave = "lienDesDiscourtTeleCharger.json"
pathFileText = "TextDiscourt"

In [5]:
thisDiscourtIscrap = dict()
methaDataToSave = dict()

# USEFUL FUNCTIONS

In [6]:
def insertText(idDoc,textContained):
    """
        use this function to create to new file for save new document haf idDoc
        @param idDoc: text file name
        @param textContained: text to save
    """
    namefile = pathFileText+"/"+idDoc+".text"
    try:
        f = open(pathFileText+"/"+idDoc+".text", "w",encoding='utf-8')
    except:
        print("Error Operation :")
        return -1
    
    try:
        f.write(textContained)
    except:
        print("Error Operation :")
        return -1
    
    f.close() 
    
    return namefile

In [25]:
def getLinkPageDiscours(linkInitialDiscourt, numberClick = 50):
    """
        use this function to get all speech links
        @param linkInitialDiscourt: is the start link
    """
    
    linkPageDiscourt = list()
    
    driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    driver.get(linkInitialDiscourt)
    
    for j in range(numberClick):
        button = driver.find_element_by_id("loadMoreNews").click()
    
    time.sleep(60)
    
    lnks = driver.find_elements_by_tag_name("a")
    
    
    for lnk in lnks:
        tampon = lnk.get_attribute("href")
        linkPageDiscourt.append(tampon)

    driver.quit()
    
    return linkPageDiscourt

In [8]:
def saveJsonFile(dictionnaryToSave, nameJsonFile):
    """
        use this function to save a dictionary as a json file
        @param dictionnaryToSave: Is the dictionary containing the data
        @param nameJsonFile: is the name of file
    """
    
    try:
        f = open(nameJsonFile, "w")
        json.dump(dictionnaryToSave, f,  indent=1)
    except:
        print("Error Operation : save file error")
        return -1
    
    f.close() 
    
    return "FINISH"

In [27]:
def createNecessaryFile():
    """
        create necessary File
    """
    fileList = [methaDataNameFile, linkDiscourtSave]
    tamponList = [pathFileText]
    
    for i in tamponList:
        try:
            os.mkdir(i)
        except OSError:
            print ("Creation of the directory %s failed" % i)
        else:
            print ("Successfully created the directory %s " % i)
            
    for j in fileList:
        if(os.path.isfile(j)==False):
            with open(j, 'w') as f:
                f.write("")
            print ("Successfully created the directory %s " % j)
        else:
            print ("Creation of the directory %s failed" % j)
        
        
    return "FINISH"

In [10]:
def openFileAsJson(nameFile):
    """
        use this function to open file as dictionnairy
    """
    dataToSave = dict()
    
    try:
        f = open(nameFile)
        try:
            if(os.path.isfile(nameFile)==True):
                dataToSave = json.load(f)
        except:
            pass
    except:
        print("Error Operation : error open file or incorrect json format")
    
    f.close()
    
    return dataToSave

In [11]:
def getDomainName(pageUrl):
    """
        give the domain name of a URL
        @param pageUrl: is the url that we want to obtain the domain name
    """
    try:
        beginIndex = pageUrl.find("//", 0, len(pageUrl))+2
        endIndex = pageUrl.find("/",beginIndex)
    except:
        return ""
    
    return pageUrl[beginIndex:endIndex]

In [12]:
def getHtmlPage(pageUrl):
    """
        give the beautifulsup object containing the html web page from the url
        @param utl: is the url of web page
    """
    try:
        requ = requests.get(pageUrl, allow_redirects=True)
    except requests.exceptions.RequestException as e:
        return BeautifulSoup("")

    return BeautifulSoup(requ.content)

In [13]:
def getLanguagePage(text):
    """
        This function detects the language of the text we have given it. It supports 55 languages
        @param text: is the text whose language we want to detect
    """
    language = ""
    try:
        language = detect(text)
    except:
        return ""
    
    return language

In [14]:
def getLink(soup, domainname):
    """
        give all link in beautiful soup html object
        return the list of link
        @param soup: is a soup variable containing html page
        @param domainname: is the domaine of web page
    """
    urlInThePage = list()
    
    if(len(soup) == 0):
        return urlInThePage
    
    for link in soup.findAll('a'):
        linkValues = link.get('href')
        if(type(linkValues)!=str or len(linkValues)==0):
            continue
        
        if(linkValues[0]=="/"):
            urlInThePage.append("http://"+domainname+linkValues)
        else:
            if("#" in linkValues):
                beginIndex = linkValues.find("#", 0, len(linkValues))
                linkValues = linkValues[0:beginIndex]
                if(len(linkValues)!=0):
                    urlInThePage.append(linkValues)
            else:
                urlInThePage.append(linkValues)

    return urlInThePage

In [15]:
def getIp(pageUrl):
    """
        return the ip address of the url pageUrl
        @param pageUrl: the url we want to get the IP address
    """
    ip=""
    pageUrl = getDomainName(pageUrl)
    try:
        ip = socket.gethostbyname(pageUrl)
    except socket.gaierror:
        ip = ""

    return ip

In [16]:
def getTextOfPage(soup):
    """
        This page extracts useful texts from your beautifulsoup object
        @param soup:
    """

    textSoup = str(soup)

    textSoup = str(soup)
    for link in soup.findAll('script'):
        textSoup = textSoup.replace(str(link)," ")

    for link in soup.findAll('style'):
        textSoup = textSoup.replace(str(link)," ")    

    beginIndex = 10
    endIndex = 100

    while(beginIndex>=0 and endIndex>=0):
        beginIndex = textSoup.find("<", 0, len(textSoup))
        endIndex = textSoup.find(">",beginIndex)+1

        textSoup = textSoup.replace(textSoup[beginIndex:endIndex],"")
    
    return textSoup

In [17]:
def getTextOfPageDiscours(DiscourtSoup):
    """
        This page extracts useful texts from your beautifulsoup object
        @param soup:
    """

    textSoup = ""
    
    textIsDiscourt = DiscourtSoup.findAll('span',{ "class" : "text-ff5000" })
    
    if(len(textIsDiscourt)!=0):
        if('Discours' in textIsDiscourt[0].text):
            textSoup = DiscourtSoup.findAll('div',{ "class" : "container-narrow mx-auto mt-50" })[0]
        
            textSoup = str(textSoup)
            for link in DiscourtSoup.findAll('script'):
                textSoup = textSoup.replace(str(link)," ")

            for link in DiscourtSoup.findAll('style'):
                textSoup = textSoup.replace(str(link)," ")    

            beginIndex = 10
            endIndex = 100

            while(beginIndex>=0 and endIndex>=0):
                beginIndex = textSoup.find("<", 0, len(textSoup))
                endIndex = textSoup.find(">",beginIndex)+1

                textSoup = textSoup.replace(textSoup[beginIndex:endIndex],"")
    
    return textSoup

In [18]:
def getTitle(soup):
    """
        we use this function to get the title
        @param soup: is a soup variable containing an html page
    """
    title=""
    try:
        title=soup.title.string
    except :
        title=""
    
    return title

# SCRAPING WEB SITE :

In [19]:
def preprocessingScrapingWebSite(urlPage):
    """
        scraping and preprocessing the web page with urlPage url
        @param urlPage: URL of the page considered
    """
    soup = getHtmlPage(urlPage)
    domainName = getDomainName(urlPage)
    ipAdress = getIp(urlPage)
    
    linkPage = getLink(soup, domainName)
    textPage = getTextOfPageDiscours(soup)
    title = getTitle(soup)
    language = getLanguagePage(textPage)
    
    return domainName, ipAdress, linkPage, title, language, textPage

# Scraping Ciblé

In [20]:
def scrapingCible(listOfUrl):
    """
        This function scrapes all the web pages of listOfUrl
        @param listOfUrl: URL list containing the URLs of the web page we want to extract
    """
    
    resultScraping = dict()
    k = 0
    for urlPage in listOfUrl:
        domainName, ipAdress, linkPage, title, language, textPage = preprocessingScrapingWebSite(urlPage)
        resultScraping[k] = {"domainName":domainName, "ipAdress":ipAdress, "urlPage":urlPage, "title":title, "language":language, "textPage":textPage}
        k+=1
        
        
    return resultScraping

# GET SPEECH FROM MACKY SALL

In [21]:
linkPage = getLinkPageDiscours(PAGE_INITIALE)



====== WebDriver manager ======
Current firefox version is 88.0
Get LATEST driver version for 88.0
Driver [/home/panongbene/.wdm/drivers/geckodriver/linux64/v0.29.1/geckodriver] found in cache


In [22]:
testest = scrapingCible(linkPage)

In [28]:
createNecessaryFile()

Successfully created the directory TextDiscourt 
Successfully created the directory InformationSurLesDiscroutEnregister.json 
Successfully created the directory lienDesDiscourtTeleCharger.json 


'FINISH'

In [29]:
k = 0
for i in testest:
    if(len(testest[i]["textPage"])>10):
        methaDataToSave[k+1] = {"domainName":testest[i]["domainName"], "ipAdress":testest[i]["ipAdress"], "urlPage":testest[i]["urlPage"], "title":testest[i]["title"], "language":testest[i]["language"], "textFile":insertText(str(k+1),testest[i]["textPage"])}
        thisDiscourtIscrap[k] = testest[i]["urlPage"]
        k=k+1

In [30]:
saveJsonFile(thisDiscourtIscrap, linkDiscourtSave)
saveJsonFile(methaDataToSave, methaDataNameFile)

'FINISH'

# UPDATE DOWNLOADS

In [54]:
methaDataNameFile = openFileAsJson(methaDataNameFile)
thisDiscourtIscrap = openFileAsJson(linkDiscourtSave)

Error Operation : error open file or incorrect json format


UnboundLocalError: local variable 'f' referenced before assignment

In [43]:
linkPage = getLinkPageDiscours(PAGE_INITIALE, 10)



====== WebDriver manager ======
Current firefox version is 88.0
Get LATEST driver version for 88.0
Driver [/home/panongbene/.wdm/drivers/geckodriver/linux64/v0.29.1/geckodriver] found in cache


In [46]:
for i in thisDiscourtIscrap:
    if(i in linkPage):
        linkPage.remove(i)

In [ ]:
testest = scrapingCible(linkPage)

In [47]:
k = len(methaDataToSave)
for i in testest:
    if(len(testest[i]["textPage"])>10):
        methaDataToSave[k+1] = {"domainName":testest[i]["domainName"], "ipAdress":testest[i]["ipAdress"], "urlPage":testest[i]["urlPage"], "title":testest[i]["title"], "language":testest[i]["language"], "textFile":insertText(str(k+1),testest[i]["textPage"])}
        thisDiscourtIscrap[k] = testest[i]["urlPage"]
        k=k+1

['http://www.presidence.sn/actualites/covid-19-respectons-les-gestes-barrieres-!_1997',
 'https://www.presidence.sn/',
 'https://www.presidence.sn/actualites',
 'https://www.presidence.sn/actualites',
 'https://www.presidence.sn/presidence/president',
 'https://www.presidence.sn/institutions',
 'https://www.presidence.sn/pse',
 'https://www.presidence.sn/monde',
 'https://www.presidence.sn/projets',
 'https://www.presidence.sn/presidence/biographie',
 'https://www.presidence.sn/pse/un-senegal-emergent',
 'https://www.presidence.sn/mediatheque',
 'https://www.presidence.sn/presse',
 'https://www.facebook.com/PresidenceSenegal/',
 'https://twitter.com/pr_senegal',
 'https://www.linkedin.com/in/prmackysall/',
 'https://www.instagram.com/pr_senegal/',
 'https://vimeo.com/presidencesn',
 'https://www.flickr.com/photos/presidencesenegal/',
 'https://www.presidence.sn/',
 'https://www.presidence.sn/en/',
 'https://www.presidence.sn/',
 'https://www.presidence.sn/actualites',
 'https://www.pre

In [ ]:
saveJsonFile(thisDiscourtIscrap, linkDiscourtSave)
saveJsonFile(methaDataToSave, methaDataNameFile)

In [51]:
len(methaDataToSave)

136